---
title: Mission Pipeline
subtitle: Mission impossible
description: Mission pipeline process data from a specific mission.
---

Generally, it includes the following steps:

- combine features from different sources/instruments (magnetic field, state data, etc.)
- generate new features

Additional components:

- `create_events_pipeline` | `create_candidate_pipeline` 
    - `create_sw_events_pipeline` : (Optional) pipeline for solar wind candidates (used in mission `JUNO` and `ARTEMIS`) to exclude events that may be influenced by planetary magnetosphere.



In [1]:
#| default_exp pipelines/default/mission

In [2]:
#| export
from discontinuitypy import PARAMS
import polars as pl
import polars.selectors as cs

from typing import Optional

## Pipelines

In [ ]:
from discontinuitypy.integration import combine_features, calc_combined_features

In [1]:
#| export
from kedro.pipeline import Pipeline, node
from kedro.pipeline.modular_pipeline import pipeline
from discontinuitypy.utils.basic import load_params

In [ ]:
# | export
def create_combined_data_pipeline(
    sat_id, # satellite id, used for namespace
    params : Optional[dict] = None,
    **kwargs
) -> Pipeline:
    
    if params is None:
        params = PARAMS
    
    tau = params["tau"]
    ts_mag = params[sat_id]["MAG"].get("time_resolution", 0)
    ts_state = params[sat_id]["STATE"].get("time_resolution", 0)
    
    ts_mag_str = f"ts_{ts_mag}s"
    ts_state_str = f"ts_{ts_state}s"
    tau_str = f"tau_{tau}s"

    node_combine_features = node(
        combine_features,
        inputs=[
            f"MAG.feature_{ts_mag_str}_{tau_str}",
            f"STATE.primary_data_{ts_state_str}",
        ],
        outputs="combined_data",
    )

    node_calc_new_features = node(
        calc_combined_features,
        inputs="combined_data",
        outputs=f"events_{ts_mag_str}_{tau_str}",
    )

    nodes = [node_combine_features, node_calc_new_features]
    return pipeline(
        nodes,
        namespace=sat_id,
        outputs={
            f"events_{ts_mag_str}_{tau_str}": f"events.{sat_id}_{ts_mag_str}_{tau_str}",
        }
    )